In [59]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE' 

import pandas as pd
import spacy
from spacy.training import Example
import multiprocessing as mp
import time

# pd.set_option('display.max_rows', None)

In [60]:
start_time = time.time()

# import labelled ner train data
train_data = pd.read_csv('train.txt', sep='\t', quoting=3)
# train_data = train_data[(train_data['comp_label'] != 'O') & (train_data['word'].str.strip() != '"')]
train_x = train_data['word'].astype(str).str.strip().tolist()
train_y = train_data['comp_label'].astype(str).str.strip().tolist()

test_data = pd.read_csv('test.txt', sep='\t', quoting=3)
# test_data = test_data[(test_data['comp_label'] != 'O') & (test_data['word'].str.strip() != '"')]
test_x = test_data['word'].astype(str).str.strip().tolist()
test_y = train_data['comp_label'].astype(str).str.strip().tolist()

Examples created with reference to documentation: https://spacy.io/api/data-formats#dict-input

In [85]:
sentences = []
labels = []
with open('train.txt') as f:
    next(f) # Skip header row
    sentence = ""
    label = []
    for i, line in enumerate(f):
        if line != "\n":
            word, man_label, comp_label = line.split("\t")
            sentence += (word + " ")
            label.append(comp_label.strip()) # Can modify
        else:
            sentences.append(sentence)
            labels.append(label)
            sentence = ""
            label = []

In [88]:
len(sentences)

9263

In [62]:
# Load the small English language model
nlp = spacy.load("en_core_web_sm")

In [63]:
# Define the function to process a chunk of data
def process_chunk(chunk):
    examples = []
    for text, label in zip(chunk[0], chunk[1]):
        examples.append(Example.from_dict(nlp.make_doc(text), {'entities': [(0, len(str(text)), label)]}))
    return examples

In [53]:
import numpy
import spacy
from spacy.attrs import ENT_IOB, ENT_TYPE

nlp = spacy.load("en_core_web_sm")
doc = nlp.make_doc("London is a big city in the United Kingdom.")
print("Before", doc.ents)  # []

header = [ENT_IOB, ENT_TYPE]
attr_array = numpy.zeros((len(doc), len(header)), dtype="uint64")
attr_array[0, 0] = 3  # B
attr_array[0, 1] = doc.vocab.strings["GPE"]
doc.from_array(header, attr_array)
print("After", doc.ents)  # [London]

Before ()
After (London,)
